<a href="https://colab.research.google.com/github/rboghe/cened/blob/master/cened_merged_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TO DO



*   take care of _x and _y


##Loading Files

In [171]:
import os
import urllib.request
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, make_scorer
from lightgbm import LGBMRegressor
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV

# Silence panda's setting with copy warning
pd.options.mode.chained_assignment = None

In [172]:
url = 'https://www.dati.lombardia.it/api/views/bbky-sde5/rows.csv?accessType=DOWNLOAD'

if os.path.isdir('/tmp'):
    if os.path.isfile('/tmp/cened20.csv'):
      pass
    else:
      urllib.request.urlretrieve(url, '/tmp/cened20.csv')
else:
  os.mkdir('/tmp')
  urllib.request.urlretrieve(url, '/tmp/cened20.csv')

In [173]:
url = 'https://www.dati.lombardia.it/api/views/rsg3-xhvk/rows.csv?accessType=DOWNLOAD'


if os.path.isfile('/tmp/cened12.csv'):
  pass
else:
  urllib.request.urlretrieve(url, '/tmp/cened12.csv')


In [174]:
url = 'https://raw.githubusercontent.com/rboghe/cened/master/degreedays.txt'

urllib.request.urlretrieve(url, '/tmp/ddh.csv')

ddh = pd.read_csv('/tmp/ddh.csv', usecols = ['comune','dd'])

In [175]:
cols20 = ['INTERO_EDIFICIO', 'RIQUALIFICAZIONE_ENERGETICA', 'ZONA_CLIMATICA',
           'ANNO_COSTRUZIONE','SUPERF_UTILE_RISCALDATA', 'SUPERF_UTILE_RAFFRESCATA',
           'VOLUME_LORDO_RISCALDATO', 'VOLUME_LORDO_RAFFRESCATO', 'CLASSE_ENERGETICA', 
           'SUPERFICIE_DISPERDENTE', 'RAPPORTO_SV','A_SOL_EST_A_SUP_UTILE','Y_IE', 
           'CLASSIFICAZIONE_DPR', 'COMUNE', 'EP_H_ND','EP_GL_NREN','EP_GL_REN', 
           'NUMERO_UNITA_IMMOBILIARI', 'RISTRUTTURAZIONE_IMPORTANTE', 'Y',
           'COMUNE','FOGLIO', 'PARTICELLA']

cols12 = ['COMUNE','FOGLIO', 'PARTICELLA','SUPERFICIE_DISPERDENTE',
          'TRASMITTANZA_MEDIA_COPERTURA','TRASMITTANZA_MEDIA_INVOLUCRO',
          'TRASMITTANZA_MEDIA_BASAMENTO','TRASMITTANZA_MEDIA_SERRAMENTO',
          'SUPERFICIE_LORDA', 'VOLUME_LORDO', 'DESTINAZIONE_DI_USO',
          'SUPERFICIE_VETRATA_OPACA', 'VOLUME_NETTO',
          'SUPERFICIE_NETTA', 'EPH'] # ANNO_COSTRUZIONE
  
cened20 = pd.read_csv('/tmp/cened20.csv', usecols = cols20)
cened12 = pd.read_csv('/tmp/cened12.csv', usecols = cols12)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Pre-filtering

In [176]:
cened20['EP_GL_NREN'].fillna(0, inplace=True)
cened20['EP_GL_REN'].fillna(0, inplace=True)

# Drop NaN 
cened20 = cened20.dropna()
cened12 = cened12.dropna() 

In [177]:
# Only entire buildings
cened20 = cened20[cened20['INTERO_EDIFICIO'] == True]

In [178]:
# Drop renovations
cened20 = cened20[cened20['RISTRUTTURAZIONE_IMPORTANTE'] == False]
cened20 = cened20[cened20['RIQUALIFICAZIONE_ENERGETICA'] == False]

## Merge

In [179]:
# Merge dfs
cenedtot = pd.merge(left=cened20, right=cened12, left_on=['COMUNE','FOGLIO', 'PARTICELLA'], 
                    right_on=['COMUNE','FOGLIO', 'PARTICELLA'],  how='inner') # , 'SUBALTERNO' | , 'SUBALTERNO'

In [180]:
# Compute difference between ext surfaces
cenedtot['surfdiff'] = abs((cenedtot['SUPERF_UTILE_RISCALDATA'] - cenedtot['SUPERFICIE_NETTA'])/cenedtot['SUPERF_UTILE_RISCALDATA'])

# Drop rows with high difference
cenedtot = cenedtot[cenedtot['surfdiff'] < 0.2]

# Compute difference between ext surfaces
#cenedtot['surfdiff'] = abs((cenedtot['SUPERFICIE_DISPERDENTE_y'] - cenedtot['SUPERFICIE_DISPERDENTE_x'])/cenedtot['SUPERFICIE_DISPERDENTE_y'])

# Drop rows with high difference
#cenedtot = cenedtot[cenedtot['surfdiff'] < 0.05]

In [181]:
# Compute difference between ext surfaces
cenedtot['ephdiff'] = abs((cenedtot['EPH'] - cenedtot['EP_H_ND'])/cenedtot['EPH'])

# Drop rows with high difference
cenedtot = cenedtot[cenedtot['ephdiff'] < 0.3]

In [182]:
cenedtot = cenedtot.dropna() 

# Preprocessing

In [183]:
# Riqualificazione
cenedtot['RIQUALIFICAZIONE_ENERGETICA'] = cenedtot['RIQUALIFICAZIONE_ENERGETICA'].astype("category").cat.codes

# Ristrutturazione
cenedtot['RISTRUTTURAZIONE_IMPORTANTE'] = cenedtot['RISTRUTTURAZIONE_IMPORTANTE'].astype("category").cat.codes

# Climatizzazione
#cenedtot['CLIMATIZZAZIONE_INVERNALE'] = cenedtot['CLIMATIZZAZIONE_INVERNALE'].astype("category").cat.codes

# Correct floats
for col in ['SUPERF_UTILE_RISCALDATA',
           'SUPERF_UTILE_RAFFRESCATA','VOLUME_LORDO_RISCALDATO','VOLUME_LORDO_RAFFRESCATO','SUPERFICIE_DISPERDENTE_x', 'SUPERFICIE_DISPERDENTE_y',
           'RAPPORTO_SV','A_SOL_EST_A_SUP_UTILE','Y_IE','EP_H_ND','EP_GL_NREN','EP_GL_REN']:
    cenedtot[col] = cenedtot[col].astype(str)
    cenedtot[col] = cenedtot[col].str.replace(r',', '')
    cenedtot[col] = cenedtot[col].astype("float")

##Feature Creation

In [184]:
# Add ddh
cenedtot['COMUNE'] = cenedtot['COMUNE'].str.lower()
ddh['comune'] = ddh['comune'].str.lower()
cenedtot['COMUNE'] = cenedtot['COMUNE'].str.replace(r"o`",  "o'")
cenedtot['COMUNE'] = cenedtot['COMUNE'].str.replace(r"baranzate",  "bollate")
cenedtot['COMUNE'] = cenedtot['COMUNE'].str.replace(r"bovisio masciago",  "bovisio-masciago")
cenedtot['COMUNE'] = cenedtot['COMUNE'].str.replace(r"cornate d`adda",  "cornate d'adda")
cenedtot['COMUNE'] = cenedtot['COMUNE'].str.replace(r"cortenuova",  "cortenova")
cenedtot['COMUNE'] = cenedtot['COMUNE'].str.replace(r"sant'omobono terme",  "sant'omobono imagna")
ddh['comune'] = ddh['comune'].str.replace(r"è",  "e'")
ddh['comune'] = ddh['comune'].str.replace(r"è",  "e'")
ddh['comune'] = ddh['comune'].str.replace(r"é",  "e'")
ddh['comune'] = ddh['comune'].str.replace(r"ò",  "o'")
ddh['comune'] = ddh['comune'].str.replace(r"ù",  "u'")

cenedtot = cenedtot.merge(ddh, left_on='COMUNE', right_on='comune', how='left')

In [185]:
cenedtot.ANNO_COSTRUZIONE = cenedtot.ANNO_COSTRUZIONE.astype(str)
cenedtot.ANNO_COSTRUZIONE = cenedtot.ANNO_COSTRUZIONE.map(lambda x:x[-4:])
cenedtot.ANNO_COSTRUZIONE = cenedtot.ANNO_COSTRUZIONE.astype(int)

cenedtot['PERIODO'] = pd.cut(cenedtot.ANNO_COSTRUZIONE, bins=[0,1930,1945,1960,1976,1992,2006,3000],
     labels=['Before 1930','1930-1945','1946-1960','1961-1976', '1977-1992', '1993-2006','After 2006']).cat.codes

In [186]:
# Opaque surface
cenedtot['SUP_OPACA'] = cenedtot['SUPERFICIE_DISPERDENTE_y']/(1+cenedtot['SUPERFICIE_VETRATA_OPACA'])

# Opaque surface
cenedtot['SUP_FINESTRATA'] = cenedtot['SUPERFICIE_DISPERDENTE_y'] - cenedtot['SUP_OPACA']

In [187]:
cenedtot['ALTEZZA_MEDIA'] = cenedtot['VOLUME_NETTO']/cenedtot['SUPERFICIE_NETTA']

In [188]:
cenedtot['TRASMITTANZA_MEDIA'] = (cenedtot['TRASMITTANZA_MEDIA_SERRAMENTO'] + cenedtot['TRASMITTANZA_MEDIA_COPERTURA'] +
         cenedtot['TRASMITTANZA_MEDIA_BASAMENTO'] + cenedtot['TRASMITTANZA_MEDIA_INVOLUCRO'])/4

In [189]:
cenedtot['EP_GL_TOT'] = cenedtot['EP_GL_REN'] + cenedtot['EP_GL_NREN']

In [190]:
# East facing surface
cenedtot['A_SOL_EST'] = cenedtot['A_SOL_EST_A_SUP_UTILE']/cenedtot['SUPERF_UTILE_RISCALDATA'] #?????????

##Filters

In [191]:
for col in ['TRASMITTANZA_MEDIA_COPERTURA','TRASMITTANZA_MEDIA_INVOLUCRO',
          'TRASMITTANZA_MEDIA_BASAMENTO','TRASMITTANZA_MEDIA_SERRAMENTO']:
          cenedtot = cenedtot[cenedtot[col].between(0,6, inclusive=False)]

In [192]:
# Drop buildings with wrong An
cenedtot = cenedtot[cenedtot['SUPERFICIE_NETTA'] > 50]

# Drop buildings with wrong Vn
cenedtot = cenedtot[cenedtot['VOLUME_NETTO'] > 150]

In [193]:
cenedtot = cenedtot[cenedtot['DESTINAZIONE_DI_USO'] == 'E.1(1)']

In [194]:
cenedtot = cenedtot[cenedtot['TRASMITTANZA_MEDIA'] > 0.15]
cenedtot = cenedtot[cenedtot['TRASMITTANZA_MEDIA'] < 4]

In [195]:
cenedtot = cenedtot[cenedtot['ALTEZZA_MEDIA'] > 2.4]
cenedtot = cenedtot[cenedtot['ALTEZZA_MEDIA'] < 6]

In [196]:
# Drop buildings with wrong EPH
# cenedtot = cenedtot[cenedtot['EP_H_ND'] > 120]
# cenedtot = cenedtot[cenedtot['EP_H_ND'] < 1000]

In [197]:
len(cenedtot)

1121

# Shuffle

In [198]:
cened = cenedtot.sample(frac=1, random_state = 42).reset_index(drop=True)

# Reserve a test set

In [199]:
msk = np.random.rand(len(cened)) < 0.8
train = cened[msk]
test = cened[~msk]

In [200]:
print(len(train))

883


In [201]:
print(len(test))

238


# Define MAPE

In [202]:
def neg_mape(y_true, y_pred):
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)
  return -np.abs((y_true - y_pred)/y_true).mean()

In [203]:
def modified_neg_mape(y_true, y_pred):
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)
  errors = np.abs((y_true - y_pred)/y_true)
  index = np.argwhere(errors > 1)
  mod_errors = np.delete(errors, index)
  return -mod_errors.mean()

In [204]:
neg_mape_scorer = make_scorer(neg_mape, greater_is_better=True)

In [205]:
modified_neg_mape_scorer = make_scorer(modified_neg_mape, greater_is_better=True)

# 2.0 inputs

## Random search

In [206]:
mlcol = ['SUPERF_UTILE_RISCALDATA',
           'VOLUME_LORDO_RISCALDATO',
           'SUPERFICIE_DISPERDENTE_x', 'dd','PERIODO',
           'RAPPORTO_SV','A_SOL_EST_A_SUP_UTILE','Y_IE',
           'ALTEZZA_MEDIA', 'RIQUALIFICAZIONE_ENERGETICA','RISTRUTTURAZIONE_IMPORTANTE',
             'A_SOL_EST', 'NUMERO_UNITA_IMMOBILIARI']

In [207]:
cat_feats = ['PERIODO']

In [208]:
param_dist = {'feature_fraction': np.linspace(0.4, 1, num=7),
              'num_leaf' : list(range(20,40)),
             'max_depth' : [-1],
             'max_bin' : [100, 200, 300, 500, 750, 1000, 2000],
             'bagging_fraction' : np.linspace(0.4, 1, num=14),
             'bagging_freq' : list(range(1,10)),
             'lambda_l1' : stats.uniform(0, 0.6),
             'lambda_l2' : stats.uniform(0, 0.6)}

lgbm = LGBMRegressor(n_estimators = 100, silent = True, verbose = 0, is_training_metric = True, n_jobs = 1, 
                     eval_metric  = 'mape')

n_iter_search = 100

random_search = RandomizedSearchCV(lgbm, param_distributions=param_dist, n_iter=n_iter_search, 
                                   scoring=modified_neg_mape_scorer, cv = 5, n_jobs = 6, verbose = 2)

random_search.fit(cened[mlcol], cened['EP_H_ND'], categorical_feature = cat_feats)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    2.8s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:   14.0s
[Parallel(n_jobs=6)]: Done 353 tasks      | elapsed:   31.5s
[Parallel(n_jobs=6)]: Done 500 out of 500 | elapsed:   44.2s finished
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['PERIODO']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=LGBMRegressor(boosting_type='gbdt',
                                           class_weight=None,
                                           colsample_bytree=1.0,
                                           eval_metric='mape',
                                           importance_type='split',
                                           is_training_metric=True,
                                           learning_rate=0.1, max_depth=-1,
                                           min_child_samples=20,
                                           min_child_weight=0.001,
                                           min_split_gain=0.0, n_estimators=100,
                                           n_jobs=1, num_leaves=31,
                                           objective=None, random_sta...
                                        'lambda_l1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f603e4d6dd8>,
            

## Print results

In [209]:
print(random_search.best_params_)

{'bagging_fraction': 0.9538461538461538, 'bagging_freq': 7, 'feature_fraction': 0.8, 'lambda_l1': 0.06240824026678044, 'lambda_l2': 0.57601269089378, 'max_bin': 2000, 'max_depth': -1, 'num_leaf': 30}


In [210]:
print(random_search.best_score_)

-0.1931873785503086


In [211]:
# Create a results df
cv_df = pd.DataFrame(random_search.cv_results_)
cv_df = cv_df.sort_values(by = ['rank_test_score']).reset_index()

# Filter columns
res_cols = [col for col in cv_df if col.startswith('split')]

# Select best iter
best_res = cv_df.loc[0]

# Print results for each fold
print(best_res[res_cols])

split0_test_score   -0.186344
split1_test_score   -0.196087
split2_test_score   -0.204112
split3_test_score    -0.18868
split4_test_score   -0.190714
Name: 0, dtype: object


## Train the final model

In [212]:
light = LGBMRegressor(n_estimators = 1000, silent = False, verbose = 2, is_training_metric = True, n_jobs = 6)
light.set_params(**random_search.best_params_)

LGBMRegressor(bagging_fraction=0.9538461538461538, bagging_freq=7,
              boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              feature_fraction=0.8, importance_type='split',
              is_training_metric=True, lambda_l1=0.06240824026678044,
              lambda_l2=0.57601269089378, learning_rate=0.1, max_bin=2000,
              max_depth=-1, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=1000, n_jobs=6, num_leaf=30,
              num_leaves=31, objective=None, random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=False, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0, verbose=2)

In [213]:
light.fit(train[mlcol], train['EP_H_ND'])

LGBMRegressor(bagging_fraction=0.9538461538461538, bagging_freq=7,
              boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              feature_fraction=0.8, importance_type='split',
              is_training_metric=True, lambda_l1=0.06240824026678044,
              lambda_l2=0.57601269089378, learning_rate=0.1, max_bin=2000,
              max_depth=-1, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=1000, n_jobs=6, num_leaf=30,
              num_leaves=31, objective=None, random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=False, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0, verbose=2)

## Test the model

In [214]:
y_pred = light.predict(test[mlcol])

In [215]:
results_df = pd.DataFrame({'predicted' : y_pred, 'true' : test['EP_H_ND']})
results_df['error'] = np.abs((results_df['true'] - results_df['predicted'])/results_df['true'])*100

In [216]:
# MAPE
results_df['error'].mean()

22.251663661092962

In [217]:
# MAPE without >100%
results_df[results_df.error <= 100].error.mean()

17.885731433296318

In [218]:
results_df

,predicted,true,error
8,84.883269,41.48,104.636619
11,102.248653,113.24,9.706240
12,151.092571,115.51,30.804754
15,182.099678,263.77,30.962703
17,72.410679,60.16,20.363496
...,...,...,...
1086,148.204919,148.82,0.413305
1089,133.404570,123.90,7.671162
1103,166.525166,161.89,2.863158
1115,138.264942,138.57,0.220147


# 1.2 inputs

## Random search

In [219]:
mlcol = ['SUPERFICIE_DISPERDENTE_y', 'SUPERFICIE_NETTA', 'VOLUME_NETTO',
          'TRASMITTANZA_MEDIA_COPERTURA','TRASMITTANZA_MEDIA_INVOLUCRO',
          'TRASMITTANZA_MEDIA_BASAMENTO','TRASMITTANZA_MEDIA_SERRAMENTO',
          'SUPERFICIE_LORDA', 'VOLUME_LORDO','dd','PERIODO','SUP_OPACA',
        'SUP_FINESTRATA']

In [220]:
cat_feats = ['PERIODO']

In [221]:
param_dist = {'feature_fraction': np.linspace(0.4, 1, num=7),
              'num_leaf' : list(range(20,40)),
             'max_depth' : [-1],
             'max_bin' : [100, 200, 300, 500, 750, 1000, 2000],
             'bagging_fraction' : np.linspace(0.4, 1, num=14),
             'bagging_freq' : list(range(1,10)),
             'lambda_l1' : stats.uniform(0, 0.6),
             'lambda_l2' : stats.uniform(0, 0.6)}

lgbm = LGBMRegressor(n_estimators = 100, silent = True, verbose = 0, is_training_metric = True, n_jobs = 1, 
                     eval_metric  = 'mape')

n_iter_search = 100

random_search = RandomizedSearchCV(lgbm, param_distributions=param_dist, n_iter=n_iter_search, 
                                   scoring=modified_neg_mape_scorer, cv = 5, n_jobs = 6, verbose = 2)

random_search.fit(cened[mlcol], cened['EP_H_ND'], categorical_feature = cat_feats)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    3.5s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:   16.3s
[Parallel(n_jobs=6)]: Done 353 tasks      | elapsed:   38.3s
[Parallel(n_jobs=6)]: Done 500 out of 500 | elapsed:   54.6s finished
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['PERIODO']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=LGBMRegressor(boosting_type='gbdt',
                                           class_weight=None,
                                           colsample_bytree=1.0,
                                           eval_metric='mape',
                                           importance_type='split',
                                           is_training_metric=True,
                                           learning_rate=0.1, max_depth=-1,
                                           min_child_samples=20,
                                           min_child_weight=0.001,
                                           min_split_gain=0.0, n_estimators=100,
                                           n_jobs=1, num_leaves=31,
                                           objective=None, random_sta...
                                        'lambda_l1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f60410374a8>,
            

## Print results

In [222]:
print(random_search.best_params_)

{'bagging_fraction': 1.0, 'bagging_freq': 8, 'feature_fraction': 0.7, 'lambda_l1': 0.030218966309218498, 'lambda_l2': 0.2242357743804975, 'max_bin': 300, 'max_depth': -1, 'num_leaf': 20}


In [223]:
print(random_search.best_score_)

-0.18203126461390934


In [224]:
# Create a results df
cv_df = pd.DataFrame(random_search.cv_results_)
cv_df = cv_df.sort_values(by = ['rank_test_score']).reset_index()

# Filter columns
res_cols = [col for col in cv_df if col.startswith('split')]

# Select best iter
best_res = cv_df.loc[0]

# Print results for each fold
print(best_res[res_cols])

split0_test_score   -0.196658
split1_test_score   -0.191859
split2_test_score   -0.176288
split3_test_score   -0.170945
split4_test_score   -0.174405
Name: 0, dtype: object


## Train the final model

In [225]:
light = LGBMRegressor(n_estimators = 1000, silent = False, verbose = 2, is_training_metric = True, n_jobs = 6)
light.set_params(**random_search.best_params_)

LGBMRegressor(bagging_fraction=1.0, bagging_freq=8, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.7,
              importance_type='split', is_training_metric=True,
              lambda_l1=0.030218966309218498, lambda_l2=0.2242357743804975,
              learning_rate=0.1, max_bin=300, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=1000, n_jobs=6, num_leaf=20, num_leaves=31,
              objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
              silent=False, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0, verbose=2)

In [226]:
light.fit(train[mlcol], train['EP_H_ND'])

LGBMRegressor(bagging_fraction=1.0, bagging_freq=8, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.7,
              importance_type='split', is_training_metric=True,
              lambda_l1=0.030218966309218498, lambda_l2=0.2242357743804975,
              learning_rate=0.1, max_bin=300, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=1000, n_jobs=6, num_leaf=20, num_leaves=31,
              objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
              silent=False, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0, verbose=2)

## Test the model

In [227]:
y_pred = light.predict(test[mlcol])

In [228]:
results_df = pd.DataFrame({'predicted' : y_pred, 'true' : test['EP_H_ND']})
results_df['error'] = np.abs((results_df['true'] - results_df['predicted'])/results_df['true'])*100

In [229]:
# MAPE
results_df['error'].mean()

18.65267452100528

In [230]:
# MAPE without >100%
results_df[results_df.error <= 100].error.mean()

17.704602733858327

In [231]:
results_df

,predicted,true,error
8,101.382720,41.48,144.413501
11,114.967201,113.24,1.525257
12,80.444098,115.51,30.357460
15,265.120528,263.77,0.512010
17,58.064016,60.16,3.484016
...,...,...,...
1086,148.823170,148.82,0.002130
1089,126.120077,123.90,1.791829
1103,203.499853,161.89,25.702547
1115,138.579991,138.57,0.007210
